<table>
<tr style="border:none;">
<td style="border:none;"><a href="https://fra.utn.edu.ar/">
<image src="https://raw.githubusercontent.com/frautn/F2-electromagnetismo/main/assets_python/logoUTN-500.svg"  width="100">
</a></td>
<td style="border:none;"><font face="times" size=4>Física 2<br>2024<br>Electromagnetismo<br>Material para trabajo en clase</font>
</td>
</tr>
</table> 

</br>

<table>
<tr style="border:none;">
<td style="border:none;"><a href="https://creativecommons.org/licenses/by-nc-sa/4.0/deed.es"><image src="https://raw.githubusercontent.com/frautn/F2-electromagnetismo/main/docs/assets/img/by-nc-sa.svg" width="70" /></a></td>
<td style="border:none;"><font face="times" size=4>2024 Edgardo Palazzo (epalazzo@fra.utn.edu.ar)</font>
</td>
</tr>
</table>

# Potencial Eléctrico

---

In [1]:
# --------------------------------
#
# En edición.
#
# --------------------------------

**Objetivos**

  - Repasar el cálculo de potencial eléctrico con cargas puntuales.
  - Analizar equipotenciales de diversas configuraciones de cargas puntuales.
  - Obtener el campo eléctrico a partir del potencial eléctrico.

**Tabla de contenidos**<a id='toc0_'></a>    
- 1. [Definición de una función para potencial eléctrico con cargas puntuales](#toc1_)    

<!-- vscode-jupyter-toc-config
	numbering=true
	anchor=true
	flat=false
	minLevel=2
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys

np.seterr(divide='ignore')  # Elimina advertencia al dividir por cero.
%precision %.3f

# ------------------------------------
# Lo siguiente es necesario para que funcione en Google Colab.
try:
  from libem import plotE, plotEf, plotEfVector, plotEfvector3d
except:
  # No descarga los archivos (aunque sí descarga .git)
  !git clone -n https://github.com/frautn/F2-electromagnetismo.git --depth 1
  !cd F2-electromagnetismo;git checkout HEAD modulos/libem.py
  sys.path.insert(0, 'F2-electromagnetismo/modulos')
  from libem import plotE, plotEf, plotEfVector, plotEfvector3d

try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

if IN_COLAB:
  !pip --quiet install ipympl
  from google.colab import output
  output.enable_custom_widget_manager()
# Fin Google colab.
# ------------------------------------

# Figuras interactivas.
# Comentar si se quiere mantener las figuras luego de cerrar el cuaderno.
# %matplotlib widget


## 1. <a id='toc1_'></a>[Definición de una función para potencial eléctrico con cargas puntuales](#toc0_)

Si consideramos que el potencial vale cero en el infinito, entonces el potencial eléctrico en una posición $\vec{\text{x}}$ en presencia de una carga puntual $q_0$ ubicada en la posición $\vec{\text{x}}_0$, puede escribirse como:
$$
V(\vec{\text{x}}) = \frac{1}{4\pi\varepsilon_0} \frac{q}{|\vec{\text{r}}|}\, , \tag{1}
$$
donde $|\vec{\text{r}}|$ es la distancia entre la posición de la partícula cargada ($\vec{\text{x}}_0$) y la posición donde se quiere determinar el valor del potencial eléctrico ($\vec{\text{x}}$).

En presencia de más de una carga eléctrica puntual, el potencial se calcula como la suma de los potenciales de cada carga:
$$
V(\vec{\text{x}}) = \frac{1}{4\pi\varepsilon_0} \sum_{i=1}^N \frac{q_i}{|\vec{\text{r}}_i|}\, , \tag{2}
$$

donde
* $q_i$ es la carga eléctrica de la partícula i-ésima 
* $|\vec{\text{r}}_i|$ es la distancia entre entre la posición donde se quiere determinar el valor del potencial eléctrico ($\vec{\text{x}}$) y la posición de la partícula cargada ($\vec{\text{x}}_i$), es decir:
$$
|\vec{\text{r}}_i| = \sqrt{(x-x_i)^2 + (y-y_i)^2 + (z-z_i)^2} ~ . \tag{3}
$$

A continuación se define una función que calcula este potencial eléctrico, siguiendo la misma metodología que en el cuaderno 01.

In [1]:
def V(x,y,z,Q):
    """Calcula potencial eléctrico en V.
    Ingresar valores de x,y,z en metros y q en coulomb.
    Q es una lista de la forma:
    Q = [
        [q1,x1,y1,z1],
        [q2,x2,y2,z2],
        ...
        [qN,xN,yN,zN]
    ]
    """
    k = 9E9   #Constante de Coulomb en las unidades correspondientes.

    V = 0
    for qi in Q:
        r = ((x - qi[1])**2 + (y - qi[2])**2 + (z - qi[3])**2)**(3/2)
        V = V + k * qi[0] / r

    return V

Calculemos el potencial de una sola partícula en alguna posición para comprobar que está funcionando correctamente.

In [3]:
Q = [
    [5E-9,0,0,0],
]

V(1,0,0,Q)

45.0

Así podemos ver las equipotenciales de este problema:

In [4]:
# Formatter para agregar V a las etiquetas de las equipotenciales.
def fmt(x):
    return f"{x}V"

In [ ]:
def equipotencialesPuntuales(cargas, dim, levels = 5, roi=(0,0,0), figsize=(6,6), titulo='Título',
                EF = False, density=1, dq=0.02):
    x = np.arange(-dim, dim+1)
    y = np.arange(-dim, dim+1)
    X, Y = np.meshgrid(x, y)
    V = x*y
    V = V*0
    for carga in cargas:
        xq, yq, q = carga
        D = np.sqrt(np.power(X - xq, 2) + 
                    np.power(Y - yq, 2))
        V = V + 9E9 * 1000 * q / D

    fig, ax = plt.subplots(1, 1, figsize=figsize,facecolor=(1, 1, 1) )
    # CS2 = ax.contour(X, Y, V, levels=levels)
    # ax.clabel(CS2, inline=True, fmt=fmt, fontsize=10)
    ax.set_title(titulo)
    for carga in cargas:
        xq, yq, q = carga
        circ = plt.Circle((xq,yq), dq*dim, color='red')
        ax.add_patch(circ)
    if roi[2]>0:
        plt.xlim((roi[0], roi[0]+roi[2]))
        plt.ylim((roi[1], roi[1]+roi[2]))
    
    if EF:
        CS2 = ax.contour(X, Y, V, levels=levels, alpha=0.4)
        E = np.gradient(-1*V)
        ax.streamplot(X, Y, E[1], E[0], linewidth=1, cmap=plt.cm.inferno,
              density=density, arrowstyle='->', arrowsize=1.5)
    else:
        CS2 = ax.contour(X, Y, V, levels=levels, alpha=1)
    ax.clabel(CS2, inline=True, fmt=fmt, fontsize=10)

    plt.xlabel('x [mm]')
    plt.ylabel('y [mm]')
    plt.grid()
    plt.show()
        
    return V